In [31]:
#package installs
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unidecode
import re

In [32]:
def statScraper(name, year, playerindex, team_id):
    index = int(playerindex)
    for x in range(2):
        url, advanced = urlConvertor(name, year, playerindex, x)
        print(url)
        try:
            result = requests.get(url)
            print(result.status_code)
            src = result.content
            soup = BeautifulSoup(src, 'lxml')
            table = soup.find('table', {'class': 'row_summable'})
            trs = table.find_all('tr')
            rows = []
            for tr in trs:
                tds = tr.find_all('td')
                row = [td.text.replace('\n', '').strip() for td in tds]
                rows.append(row)
            if x == 0:
                columns = ['G', 'Date', 'Age', 'Tm', 'Location', 'Opp',
                           'GS', 'Active', 'MP', 'FG', 'FGA', 'FG%',
                           '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
                           'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
                           'PTS', 'GmSc', '+/-']
                df = pd.DataFrame(rows, columns=columns)
                df = df[df['Tm'] == team_id]
                if len(df) == 0:
                    index += 1
                    newindex = '0' + str(index)
                    statScraper(name, year, newindex, team_id)
                result.close()
                df['PlayerID'] = name[2]
            elif x == 1:
                columns = ['G', 'Date', 'Age', 'Tm', 'Location', 'Opp', 'GS',
                           'Active', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%',
                           'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
                           'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM']
                df2 = pd.DataFrame(rows, columns=columns)
                df2 = df2[df2['Tm'] == team_id]
                if len(df2) == 0:
                    index += 1
                    newindex = '0' + str(index)
                    statScraper(name, year, newindex, team_id)
                result.close()
                df2 = df2.drop(df2.columns[[0, 2, 3, 4, 5, 6, 7, 8, -2]], axis=1)
        except:
            print('Error Scraping' ,index)
            index+=1
            if index > 4:
                print('Stat scraping failed for:', name[0], name[1])
                print(url)
                return None
            newindex = '0' + str(index)
            statScraper(name, year, newindex, team_id)
    statdf = pd.merge(df, df2,how='outer', on='Date')
    statdf.insert(0, 'Name', name[1] + ', ' + name[0])
    print('Successfully Scraped:', name[0], name[1])
    return statdf


In [34]:
def urlConvertor(name, year, playerindex, advanced: bool):
    lname = name[1]
    fname = name[0]
    lnamelist = list(lname)
    linitial = lnamelist[0]
    fnamelist = list(fname)
    lname5 = ''
    for x in range(5):
        try:
            lname5 += lnamelist[x]
        except:
            break
    f2i = fnamelist[0] + fnamelist[1]
    if advanced == False:
        url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog/{year}/'
    else:
        url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog-advanced/{year}/'
    return url, advanced


In [36]:
def rosterScraper(TeamID, year, id_number):
    url = f"https://www.basketball-reference.com/teams/{TeamID}/{year}.html"
    result = requests.get(url)
    try:
        src = result.content
        soup = BeautifulSoup(src, 'lxml')
        table = soup.find('table', {'class': 'sortable'})
        names = []
        trs = table.find_all('tr')
        rows = []
        for tr in trs:
            try:
                td = tr.find('td', {'data-stat': 'player'})
                a_tag = td.find('a')
                name = a_tag.get_text()
                name = unidecode.unidecode(name)
                name = re.sub(r'[^\w\s]','',name)
                names.append(name)
            except:
                pass
        for x in range(len(names)):
            names[x] = names[x].split()
            names[x].append(id_number)
            id_number += 1
        print(TeamID,"roster successfully parsed")
        return names
    except:
        return False



In [37]:
def teamFinder(year):
    team_id = 101
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')
    table = soup.find('table', {'class': 'stats_table sortable', 'data-cols-to-freeze': ',2', 'id': 'per_game-team'})
    teams = []
    ids = {}
    trs = table.find_all('tr')
    x = 0
    for tr in trs[1:]:
        #print(tr.prettify())
        try:
            td = tr.find('td', {'data-stat': 'team', 'class': 'left'})
            a_tag = td.find('a')
            team = a_tag.get_text()
            teams.append(team)
            link = a_tag.attrs['href']
            link = list(link)
            acronym = link[7] + link[8] + link[9]
            ids[teams[x]] = []
            ids[teams[x]].append(acronym)
            x+=1
        except:
            pass
    for x in range(len(teams)):
        ids[teams[x]].append(team_id)
        team_id+=1
    return teams, ids

In [38]:
def leagueRosters(year):
    teamrosters = {}
    id_number = 1001
    teams, team_ids = teamFinder(year)
    for team in teams:
        teamrosters[team] = rosterScraper(team_ids[team][0], year, id_number)
        id_number+=len(teamrosters[team])
    return teamrosters, team_ids

In [39]:
def teamRoster(team, year):
    roster = []
    id_number = 1001
    teams, team_ids = teamFinder(year)
    roster = rosterScraper(team_ids[team][0], year, id_number)
    return roster, team_ids


In [40]:
def playerScraper(name, team, year):
    name = name.split()
    name.append('1001')
    teams, ids = teamFinder(year)
    team_id = ids[team][0]
    df = statScraper(name, year, '01', team_id)
    return df


In [41]:
def teamScraper(team, year):
    roster, team_ids = teamRoster(team, year)
    for x in range(len(roster)):
        try:
            if x == 0:
                player = roster[x]
                df = statScraper(player, year, '01', team_ids[team][0])
            else:
                player = roster[x]
                df = df.append(statScraper(player, year, '01', team_ids[team][0]))
        except:
            pass
    return df

In [42]:
def leagueScraper(year):
    rosters, team_ids = leagueRosters(year)
    y = 0
    for team in rosters:
        for x in range(len(rosters[team])):
            try:
                if x == 0 and y == 0:
                    player = rosters[team][x]
                    print(player)
                    df = statScraper(player, year, '01', team_ids[team][0])
                else:
                    player = rosters[team][x]
                    df = df.append(statScraper(player, year, '01', team_ids[team][0]))
            except:
                pass
        y+=1
    return df

In [43]:
def save_df(df, title, path):
    df.to_csv(rf'{path}{title}.csv', index=False)


In [46]:
teamScraper('Orlando Magic', '2002')

ORL roster successfully parsed
https://www.basketball-reference.com/players/A/ArmstDa01/gamelog/2002/
200
https://www.basketball-reference.com/players/A/ArmstDa01/gamelog-advanced/2002/
200
Successfully Scraped: Darrell Armstrong
https://www.basketball-reference.com/players/B/BrownDe01/gamelog/2002/
200
https://www.basketball-reference.com/players/B/BrownDe01/gamelog-advanced/2002/
200
Successfully Scraped: Dee Brown
https://www.basketball-reference.com/players/B/BuechJu01/gamelog/2002/
200
https://www.basketball-reference.com/players/B/BuechJu01/gamelog-advanced/2002/
200
Successfully Scraped: Jud Buechler
https://www.basketball-reference.com/players/D/DeCleAn01/gamelog/2002/
200
https://www.basketball-reference.com/players/D/DeCleAn01/gamelog-advanced/2002/
200
Successfully Scraped: Andrew DeClercq
https://www.basketball-reference.com/players/E/EwingPa01/gamelog/2002/
200
https://www.basketball-reference.com/players/E/EwingPa01/gamelog-advanced/2002/
200
Successfully Scraped: Patrick

,Name,G,Date,Age,Tm,Location,Opp,GS,Active,MP,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,"Armstrong, Darrell",1,2001-10-30,33-130,ORL,,TOR,W (+29),1,28:42,...,10.1,8.8,24.3,5.5,2.6,11.3,14.3,185,89,21.7
1,"Armstrong, Darrell",2,2001-11-02,33-133,ORL,@,IND,L (-10),1,30:00,...,8.6,7.9,27.1,3.5,0.0,0.0,27.7,140,125,10.2
2,"Armstrong, Darrell",3,2001-11-03,33-134,ORL,@,CLE,W (+30),1,24:19,...,17.2,9.2,36.4,6.5,2.9,0.0,14.2,129,63,14.1
3,"Armstrong, Darrell",4,2001-11-05,33-136,ORL,,SEA,L (-4),1,44:18,...,13.2,9.5,21.3,2.5,0.0,0.0,15.6,144,115,7.5
4,"Armstrong, Darrell",5,2001-11-06,33-137,ORL,@,SAS,L (-15),1,33:50,...,11.8,5.0,23.9,0.0,0.0,11.4,10.4,105,121,-4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,"Williams, Monty",64,2002-04-11,30-185,ORL,,NJN,L (-2),0,13:45,...,9.0,4.1,11.7,0.0,0.0,40.0,16.2,79,104,-11.8
67,"Williams, Monty",65,2002-04-12,30-186,ORL,@,MIA,L (-5),0,11:49,...,18.6,8.3,0.0,12.6,0.0,0.0,6.8,62,79,2.1
68,"Williams, Monty",66,2002-04-14,30-188,ORL,,PHI,L (-6),0,11:09,...,12.8,10.9,51.8,5.2,0.0,11.1,40.7,69,108,-6.0
69,"Williams, Monty",67,2002-04-16,30-190,ORL,@,ATL,W (+7),1,37:10,...,14.4,10.0,15.0,2.6,1.8,9.9,23.7,118,92,6.3
